### The case study is from an open source dataset from Kaggle. 

##### Link to the Kaggle project site:

https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling (Links to an external site.)

### Given a Bank customer, can we build a classifier which can determine whether they will leave or not using Neural networks?

#### 1. Read the dataset

In [0]:
import numpy as np
import pandas as pd

bank = pd.read_csv("bank.csv")

In [2]:
bank.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


#### 2. Drop the columns which are unique for all users like IDs

In [0]:
bank.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [6]:
bank.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
bank["Gender"], mapping_index = pd.factorize(bank["Gender"])
bank.sample(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
2884,620,France,0,29,3,0.00,2,0,1,153392.28,0
2365,701,Germany,1,32,5,102500.34,1,0,0,106287.77,0
457,711,France,1,38,3,0.00,2,1,0,68487.51,0
4691,622,Spain,1,37,4,0.00,2,1,0,4459.50,0
1676,812,Spain,0,44,8,0.00,3,1,0,66926.83,1


In [9]:
bank_num = pd.get_dummies(bank, columns=["Geography"], prefix=["Geography"], dtype='int64')
bank_num.sample(10).T

,8940,1894,5643,838,7255,8012,9353,2073,7937,1591
CreditScore,657.00,535.00,616.00,553.00,563.00,747.0,670.00,626.00,670.00,621.00
Gender,1.00,0.00,0.00,0.00,0.00,0.0,1.00,0.00,1.00,1.00
Age,38.00,49.00,36.00,48.00,32.00,34.0,37.00,48.00,36.00,39.00
Tenure,7.00,7.00,6.00,3.00,0.00,10.0,0.00,2.00,3.00,6.00
Balance,0.00,115309.75,0.00,0.00,148326.09,0.0,178742.71,0.00,0.00,0.00
NumOfProducts,2.00,1.00,1.00,1.00,1.00,2.0,1.00,2.00,1.00,2.00
HasCrCard,1.00,1.00,1.00,0.00,1.00,1.0,1.00,1.00,1.00,1.00
IsActiveMember,0.00,0.00,1.00,1.00,0.00,1.0,1.00,1.00,0.00,1.00
EstimatedSalary,185827.74,111421.77,12916.32,30730.95,191604.27,50759.8,194493.57,95794.98,140754.19,58883.91
Exited,0.00,0.00,1.00,1.00,1.00,0.0,0.00,0.00,1.00,0.00


#### 3. Distinguish the feature and target set

In [0]:
X = bank_num.drop(['Exited'], axis=1)
Y = bank_num['Exited']

#### 4.Divide the data set into Train and test sets

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

#### 5. Normalize the train and test data

In [0]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

#### 6. Initialize & build the model

In [33]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_acc', patience=5)
model = Sequential()
model.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
model.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics = ['accuracy'])
model.fit(x_train,y_train, validation_data=(x_test, y_test), epochs=100, batch_size=20, callbacks=[es])

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 3s 351us/step - loss: 0.5490 - acc: 0.7960 - val_loss: 0.4411 - val_acc: 0.7975
Epoch 2/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.4230 - acc: 0.7960 - val_loss: 0.4061 - val_acc: 0.7975
Epoch 3/100
8000/8000 [==============================] - 1s 95us/step - loss: 0.3990 - acc: 0.7960 - val_loss: 0.3905 - val_acc: 0.7975
Epoch 4/100
8000/8000 [==============================] - 1s 94us/step - loss: 0.3853 - acc: 0.8005 - val_loss: 0.3784 - val_acc: 0.8450
Epoch 5/100
8000/8000 [==============================] - 1s 93us/step - loss: 0.3792 - acc: 0.8430 - val_loss: 0.3750 - val_acc: 0.8415
Epoch 6/100
8000/8000 [==============================] - 1s 92us/step - loss: 0.3754 - acc: 0.8458 - val_loss: 0.3751 - val_acc: 0.8440
Epoch 7/100
8000/8000 [==============================] - 1s 92us/step - loss: 0.3733 - acc: 0.8464 - val_loss: 0.3685 - val_acc: 0.8470

#### 7. Optimize the model (Optional)

In [39]:
from keras.layers.normalization import BatchNormalization

model = Sequential()
model.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
model.add(BatchNormalization())
model.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
model.add(BatchNormalization())
model.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
model.add(BatchNormalization())
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics = ['accuracy'])
model.fit(x_train,y_train, validation_data=(x_test, y_test), epochs=100, batch_size=20, callbacks=[es])

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 5s 579us/step - loss: 0.5501 - acc: 0.7565 - val_loss: 0.3999 - val_acc: 0.8530
Epoch 2/100
8000/8000 [==============================] - 1s 137us/step - loss: 0.3955 - acc: 0.8368 - val_loss: 0.3404 - val_acc: 0.8600
Epoch 3/100
8000/8000 [==============================] - 1s 140us/step - loss: 0.3759 - acc: 0.8440 - val_loss: 0.3520 - val_acc: 0.8535
Epoch 4/100
8000/8000 [==============================] - 1s 139us/step - loss: 0.3713 - acc: 0.8458 - val_loss: 0.3481 - val_acc: 0.8570
Epoch 5/100
8000/8000 [==============================] - 1s 138us/step - loss: 0.3687 - acc: 0.8464 - val_loss: 0.3534 - val_acc: 0.8585
Epoch 6/100
8000/8000 [==============================] - 1s 144us/step - loss: 0.3663 - acc: 0.8486 - val_loss: 0.3463 - val_acc: 0.8625
Epoch 7/100
8000/8000 [==============================] - 1s 147us/step - loss: 0.3627 - acc: 0.8505 - val_loss: 0.3374 - val_acc: 

#### 8. Predict the results using 0.5 as a threshold (Optional)

In [0]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

#### 9.Print the Accuracy score and confusion matrix

In [42]:
scores = model.evaluate(x_train, y_train)
scores[1]*100

8000/8000 [==============================] - 0s 43us/step


86.2125

In [43]:
scores = model.evaluate(x_test, y_test)
scores[1]*100

2000/2000 [==============================] - 0s 43us/step


86.35000000000001

In [44]:
from sklearn import metrics
print('Accuracy: ' , metrics.accuracy_score(y_test,y_pred))

Accuracy:  0.8635


In [45]:
report=metrics.classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.88      0.95      0.92      1595
           1       0.74      0.50      0.60       405

    accuracy                           0.86      2000
   macro avg       0.81      0.73      0.76      2000
weighted avg       0.85      0.86      0.85      2000



In [46]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, y_pred)
print (confusion)

[[1523   72]
 [ 201  204]]
